<a href="https://colab.research.google.com/github/huunghia160799/CS224W-Stanford/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from functools import reduce

n = 5
phi = [[None for i in range(n + 1)] for i in range(n + 1)]

phi[1][2] = phi[3][4] = np.array([[1, 0.9], [0.9, 1]])
phi[2][3] = phi[3][5] = np.array([[0.1, 1], [1, 0.1]])
phi[2][2] = phi[4][4] = np.array([[1, 0.1], [0.1, 1]])

belief = [None for i in range(n + 1)]

# term = phi[1][2].dot(phi[2][3].dot(phi[3][4].dot(phi[4][4])) + phi[2][3].dot(phi[3][5]))
# term = reduce(np.dot, [phi[1][2], phi[2][3], phi[3][4], phi[4][4], phi[3][5]])
# term

post_3 = phi[3][4].dot(phi[4][4].dot(np.array([0, 1])) + phi[3][5].dot(np.array([1, 1])))
post_2 = phi[2][2].dot(np.array([1, 0])) + phi[2][3].dot(post_3)
unnormalized = phi[1][2].dot(post_2)
belief[1] = unnormalized / np.sum(unnormalized)

unnormalized = phi[1][2].T.dot(np.ones(2)) + phi[2][3].dot(post_3)
belief[2] = unnormalized / np.sum(unnormalized)

post_2 = phi[1][2].T.dot(np.ones(2)) + phi[2][2].dot(np.array([1, 0]))
unnormalized = phi[3][4].dot(phi[4][4].dot(np.array([0, 1])) + phi[3][5].dot(np.array([1, 1]))) + phi[2][3].T.dot(post_2)
belief[3] = unnormalized / np.sum(unnormalized)

post_3 = phi[2][3].T.dot(post_2) + phi[3][4].dot(phi[4][4].dot(np.array([0, 1])))
unnormalized = phi[3][5].T.dot(post_3)
belief[5] = unnormalized / np.sum(unnormalized)

post_3 = phi[2][3].T.dot(post_2) + phi[3][5].dot(np.ones(2))
unnormalized = phi[3][4].T.dot(post_3)
belief[4] = unnormalized / np.sum(unnormalized)

for i in range(1, n + 1):
    print("{}: {}".format(i, belief[i]))

1: [0.50322818 0.49677182]
2: [0.50378611 0.49621389]
3: [0.46140652 0.53859348]
4: [0.4971916 0.5028084]
5: [0.54922217 0.45077783]


# 4. GNN Implementation

In [2]:
%cd /content/drive/My Drive/CS224W Stanford: Machine Learning in Graphs/hw2/q4_starter_code
!pwd

/content/drive/My Drive/CS224W Stanford: Machine Learning in Graphs/hw2/q4_starter_code
/content/drive/My Drive/CS224W Stanford: Machine Learning in Graphs/hw2/q4_starter_code


In [3]:
!pip install torch torch-geometric seaborn
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 235kB 8.4MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=e3d6d2296de5eae8459829644f506904a3ff441f421cc93f0bfc32bdf6488821
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
Successfully built torch-geometric
Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 1.0MB/s 
     |████████████████████████████████| 15.2MB 209kB/s 


In [4]:
%%writefile utils.py

import torch.optim as optim

def parse_optimizer(parser):
    opt_parser = parser.add_argument_group()
    opt_parser.add_argument('--opt', dest='opt', type=str,
            help='Type of optimizer')
    opt_parser.add_argument('--opt-scheduler', dest='opt_scheduler', type=str,
            help='Type of optimizer scheduler. By default none')
    opt_parser.add_argument('--opt-restart', dest='opt_restart', type=int,
            help='Number of epochs before restart (by default set to 0 which means no restart)')
    opt_parser.add_argument('--opt-decay-step', dest='opt_decay_step', type=int,
            help='Number of epochs before decay')
    opt_parser.add_argument('--opt-decay-rate', dest='opt_decay_rate', type=float,
            help='Learning rate decay ratio')
    opt_parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
    opt_parser.add_argument('--clip', dest='clip', type=float,
            help='Gradient clipping.')
    opt_parser.add_argument('--weight_decay', type=float,
            help='Optimizer weight decay.')

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

Overwriting utils.py


## models.py

In [14]:
%%writefile models.py

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch_geometric.utils import add_self_loops, degree

import inspect


class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, model_type, task='node'):
        super(GNNStack, self).__init__()
        conv_model = self.build_conv_model(model_type)
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(hidden_dim, hidden_dim))

        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(args.dropout), 
            nn.Linear(hidden_dim, output_dim))

        self.task = task
        if not (self.task == 'node' or self.task == 'graph'):
            raise RuntimeError('Unknown task.')

        self.dropout = args.dropout
        self.num_layers = args.num_layers

    def build_conv_model(self, model_type):
        if model_type == 'GCN':
            return pyg_nn.GCNConv
        elif model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        ############################################################################
        # TODO: Your code here! 
        # Each layer in GNN should consist of a convolution (specified in model_type),
        # a non-linearity (use RELU), and dropout. 
        # HINT: the __init__ function contains parameters you will need. You may 
        # also find pyg_nn.global_max_pool useful for graph classification.
        # Our implementation is ~6 lines, but don't worry if you deviate from this.

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        if self.task == 'graph':
            x = pyg_nn.global_mean_pool(x, batch)

        ############################################################################

        x = self.post_mp(x)

        return F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)


class GraphSage(pyg_nn.MessagePassing):
    """Non-minibatch version of GraphSage."""
    def __init__(self, in_channels, out_channels, reducer='mean', 
                 normalize_embedding=True):
        super(GraphSage, self).__init__(aggr='mean')

        ############################################################################
        # TODO: Your code here! 
        # Define the layers needed for the forward function. 
        # Our implementation is ~2 lines, but don't worry if you deviate from this.

        self.lin = nn.Linear(in_channels, out_channels)
        self.agg_lin = nn.Linear(in_channels, out_channels)

        ############################################################################

        if normalize_embedding:
            self.normalize_emb = True

    def forward(self, x, edge_index):
        num_nodes = x.size(0)
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        ############################################################################
        # TODO: Your code here! 
        # Given x, perform the aggregation and pass it through a MLP with skip-
        # connection. Place the result in out. 
        # HINT: It may be useful to read the pyg_nn implementation of GCNConv,
        # https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html
        # Our implementation is ~4 lines, but don't worry if you deviate from this.

        # residual = x
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        self_x = self.agg_lin(x)
        # x = F.relu(x)
        # out = x

        ############################################################################
        
        return self_x + self.propagate(edge_index, size=num_nodes, x=self.lin(x))

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        row, col = edge_index
        deg = pyg_utils.degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        ############################################################################
        # TODO: Your code here! Perform the update step here. 
        # Our implementation is ~1 line, but don't worry if you deviate from this.

        # if self.normalize_emb:
            # aggr_out = F.normalize(aggr_out, p=2, dim=-1)

        ############################################################################

        return aggr_out


class GAT(pyg_nn.MessagePassing):

    def __init__(self, in_channels, out_channels, num_heads=1, concat=True,
                 dropout=0, bias=True, **kwargs):
        super(GAT, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = num_heads
        self.concat = concat 
        self.dropout = dropout

        ############################################################################
        #  TODO: Your code here!
        # Define the layers needed for the forward function. 
        # Remember that the shape of the output depends the number of heads.
        # Our implementation is ~1 line, but don't worry if you deviate from this.

        self.lin = nn.Linear(in_channels, self.heads * out_channels)

        ############################################################################

        ############################################################################
        #  TODO: Your code here!
        # The attention mechanism is a single feed-forward neural network parametrized
        # by weight vector self.att. Define the nn.Parameter needed for the attention
        # mechanism here. Remember to consider number of heads for dimension!
        # Our implementation is ~1 line, but don't worry if you deviate from this.

        self.att = nn.Parameter(torch.Tensor(1, self.heads, 2 * out_channels))

        ############################################################################

        if bias and concat:
            self.bias = nn.Parameter(torch.Tensor(self.heads * out_channels))
        elif bias and not concat:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        nn.init.xavier_uniform_(self.att)
        nn.init.zeros_(self.bias)

        ############################################################################

    def forward(self, x, edge_index, size=None):
        ############################################################################
        #  TODO: Your code here!
        # Apply your linear transformation to the node feature matrix before starting
        # to propagate messages.
        # Our implementation is ~1 line, but don't worry if you deviate from this.
        
        x = self.lin(x)
        ############################################################################

        # Start propagating messages.
        return self.propagate(edge_index, size=size, x=x)

    def message(self, edge_index_i, x_i, x_j, size_i):
        #  Constructs messages to node i for each edge (j, i).

        ############################################################################
        #  TODO: Your code here! Compute the attention coefficients alpha as described
        # in equation (7). Remember to be careful of the number of heads with 
        # dimension!
        # Our implementation is ~5 lines, but don't worry if you deviate from this.

        x_j = x_j.view(-1, self.heads, self.out_channels)
        x_i = x_i.view(-1, self.heads, self.out_channels)
        alpha = (torch.cat((x_i, x_j), dim=-1) * self.att).sum(dim=-1)
        alpha = F.leaky_relu(alpha)
        alpha = pyg_utils.softmax(alpha, edge_index_i, size_i)

        ############################################################################

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        return x_j * alpha.view(-1, self.heads, 1)

    def update(self, aggr_out):
        # Updates node embedings.
        if self.concat is True:
            aggr_out = aggr_out.view(-1, self.heads * self.out_channels)
        else:
            aggr_out = aggr_out.mean(dim=1)

        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

Overwriting models.py


## train.py

In [16]:
%%writefile train.py

import argparse
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

import models
import utils


def arg_parse():
    parser = argparse.ArgumentParser(description='GNN arguments.')
    utils.parse_optimizer(parser)

    parser.add_argument('--model_type', type=str,
                        help='Type of GNN model.')
    parser.add_argument('--batch_size', type=int,
                        help='Training batch size')
    parser.add_argument('--num_layers', type=int,
                        help='Number of graph conv layers')
    parser.add_argument('--hidden_dim', type=int,
                        help='Training hidden size')
    parser.add_argument('--dropout', type=float,
                        help='Dropout rate')
    parser.add_argument('--epochs', type=int,
                        help='Number of training epochs')
    parser.add_argument('--dataset', type=str,
                        help='Dataset')

    parser.set_defaults(model_type='GCN',
                        dataset='cora',
                        num_layers=2,
                        batch_size=32,
                        hidden_dim=32,
                        dropout=0.0,
                        epochs=200,
                        opt='adam',   # opt_parser
                        opt_scheduler='none',
                        weight_decay=0.0,
                        lr=0.01)

    return parser.parse_args()

def train(dataset, task, args, model_type):
    if task == 'graph':
        # graph classification: separate dataloader for test set
        data_size = len(dataset)
        loader = DataLoader(
                dataset[:int(data_size * 0.8)], batch_size=args.batch_size, shuffle=True)
        test_loader = DataLoader(
                dataset[int(data_size * 0.8):], batch_size=args.batch_size, shuffle=True)
    elif task == 'node':
        # use mask to split train/validation/test
        test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    else:
        raise RuntimeError('Unknown task')

    # build model
    model = models.GNNStack(dataset.num_node_features, args.hidden_dim, dataset.num_classes, 
                            args, model_type, task=task)
    scheduler, opt = utils.build_optimizer(args, model.parameters())

    # train
    accuracies = []
    epoch_num = []
    for epoch in range(args.epochs):
        total_loss = 0
        model.train()
        for batch in loader:
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            if task == 'node':
                pred = pred[batch.train_mask]
                label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        # print(total_loss)

        if epoch % 10 == 0:
            test_acc = test(loader, model)
            # print(test_acc,   '  test')
            accuracies.append(test_acc)
            epoch_num.append(epoch)
    
    return accuracies, epoch_num
        

def test(loader, model, is_validation=False):
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            pred = model(data).max(dim=1)[1]
            label = data.y

        if model.task == 'node':
            mask = data.val_mask if is_validation else data.test_mask
            # node classification: only evaluate on nodes in test set
            pred = pred[mask]
            label = data.y[mask]
            
        correct += pred.eq(label).sum().item()
    
    if model.task == 'graph':
        total = len(loader.dataset) 
    else:
        total = 0
        for data in loader.dataset:
            total += torch.sum(data.test_mask).item()
    return correct / total

def main():
    args = arg_parse()

    if args.dataset == 'enzymes':
        dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
        task = 'graph'
    elif args.dataset == 'cora':
        dataset = Planetoid(root='/tmp/Cora', name='Cora')
        task = 'node'
    
    print('GCN')
    acc_gcn, epoch_num = train(dataset, task, args, 'GCN')
    # print('GraphSage')
    # acc_gsage, _ = train(dataset, task, args, 'GraphSage')
    print('GAT')
    acc_gat, _ = train(dataset, task, args, 'GAT')
    visualization = pd.DataFrame({
        "epoch": epoch_num,
        "gcn": acc_gcn,
        # "graphsage": acc_gsage,
        "gat": acc_gat
    })
    ax = sns.lineplot(x="epoch", y="value", hue="variable", data=pd.melt(visualization, ['epoch']))
    file_path = task + '.png'
    print(file_path)
    ax.figure.savefig(file_path)

if __name__ == '__main__':
    main()


Overwriting train.py


## Running run.sh

In [17]:
!bash run.sh

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
GCN
GAT
graph.png


In [8]:
%%writefile run.sh
# python train.py --dataset=cora --dropout=0.5 --weight_decay=5e-3 --epochs=500
python train.py --dataset=enzymes --weight_decay=5e-3 --num_layers=3 --epochs=500

Overwriting run.sh
